Importing pandas ; Importing book value csv

In [1]:
import pandas as pd
df=pd.read_csv('../../data/input/book-value.csv',sep=';')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Year        16 non-null     int64  
 1   Book Value  16 non-null     float64
 2   PBV         16 non-null     float64
dtypes: float64(2), int64(1)
memory usage: 516.0 bytes
None


Setting Year column as index ; Creating Growth column ; Creating x year growth column

In [2]:
df=df.set_index('Year')
df['1 Growth'] = df['Book Value'].diff()/df['Book Value'].shift(1)

growth_mean_interval=range(2,6)
for i in growth_mean_interval:
    df[str(i)+' Mean Growth'] = df['1 Growth'].rolling(window=i).mean()

print(df.head(len(df)))


      Book Value   PBV  1 Growth  2 Mean Growth  3 Mean Growth  4 Mean Growth  \
Year                                                                            
2008     188.800  3.44       NaN            NaN            NaN            NaN   
2009     226.000  4.29  0.197034            NaN            NaN            NaN   
2010     276.680  4.63  0.224248       0.210641            NaN            NaN   
2011     340.726  4.70  0.231480       0.227864       0.217587            NaN   
2012     420.416  4.33  0.233883       0.232682       0.229870       0.221661   
2013     518.074  3.71  0.232289       0.233086       0.232551       0.230475   
2014     630.160  4.17  0.216351       0.224320       0.227508       0.228501   
2015     724.960  3.67  0.150438       0.183395       0.199693       0.208240   
2016     912.050  3.40  0.258069       0.204254       0.208286       0.214287   
2017    1065.126  4.11  0.167837       0.212953       0.192115       0.198174   
2018    1230.254  4.23  0.15

Creating price prediction dataframe ; Dropping unrelevant columns

In [3]:
df_price=df
df_price.index=df_price.index+1

for i in growth_mean_interval:
    df_price[str(i)+' Prediction'] = (1 + df_price[str(i)+' Mean Growth'])*df_price['Book Value']*df_price['PBV']

for i in growth_mean_interval:
    df_price.drop(columns=[str(i)+' Mean Growth'],inplace=True)

df_price=df_price.drop(columns=['1 Growth','Book Value','PBV'])
print(df_price.head())

      2 Prediction  3 Prediction  4 Prediction  5 Prediction
Year                                                        
2009           NaN           NaN           NaN           NaN
2010           NaN           NaN           NaN           NaN
2011   1550.865302           NaN           NaN           NaN
2012   1966.316548   1949.859262           NaN           NaN
2013   2243.975303   2238.857615   2223.913722           NaN


Exporting price prediction by book value growth csv

In [4]:
df_price=df_price.dropna()
df_price.to_csv('../../data/output/price-prediction-by-book-value-growth.csv', index=True)